In [14]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium as fl

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
print( tabulate(df[0], headers='keys', tablefmt='psql') )
 

+-----+------------+------------------+---------------------------------------------------+
|     | Postcode   | Borough          | Neighbourhood                                     |
|-----+------------+------------------+---------------------------------------------------|
|   0 | M1A        | Not assigned     | Not assigned                                      |
|   1 | M2A        | Not assigned     | Not assigned                                      |
|   2 | M3A        | North York       | Parkwoods                                         |
|   3 | M4A        | North York       | Victoria Village                                  |
|   4 | M5A        | Downtown Toronto | Harbourfront                                      |
|   5 | M6A        | North York       | Lawrence Heights                                  |
|   6 | M6A        | North York       | Lawrence Manor                                    |
|   7 | M7A        | Queen's Park     | Not assigned                            

In [88]:
df[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [89]:
na_rows = df[0].Borough == "Not assigned"
f0 = df[0][~na_rows]

In [90]:
pd.set_option('mode.chained_assignment', None)
nb_rows = f0.Neighbourhood == "Not assigned"
f0.loc[nb_rows, 'Neighbourhood'] = f0.loc[nb_rows, 'Borough']
pd.set_option('mode.chained_assignment', 'warn')

In [91]:
grouped = f0.groupby('Postcode')
ndf = pd.DataFrame(columns=f0.columns)

In [92]:
for postcode, group in grouped:
    g = {}
    g['Postcode'] = postcode
    g['Borough'] = group.Borough.iloc[0]
    g['Neighbourhood'] = ",".join(group['Neighbourhood'].values.tolist())
    ndf = ndf.append(g, ignore_index=True)

In [93]:
print(ndf.shape)

(103, 3)


In [94]:
df_g=pd.read_csv('http://cocl.us/Geospatial_data')

In [95]:
df_g.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
df_g.rename(columns={'Postal Code':'PostalCode'}, inplace = True)

In [107]:
j = ndf.merge(df_g,left_on='Postcode', right_on='PostalCode')

In [109]:
final = j.drop('PostalCode', axis=1)
print(final.head(10))

  Postcode      Borough                                  Neighbourhood  \
0      M1B  Scarborough                                  Rouge,Malvern   
1      M1C  Scarborough           Highland Creek,Rouge Hill,Port Union   
2      M1E  Scarborough                Guildwood,Morningside,West Hill   
3      M1G  Scarborough                                         Woburn   
4      M1H  Scarborough                                      Cedarbrae   
5      M1J  Scarborough                            Scarborough Village   
6      M1K  Scarborough      East Birchmount Park,Ionview,Kennedy Park   
7      M1L  Scarborough                  Clairlea,Golden Mile,Oakridge   
8      M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West   
9      M1N  Scarborough                     Birch Cliff,Cliffside West   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  
5  43.744734 -79.239476  
6  

In [118]:
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [127]:
kclusters = 5
k = final.drop(['Borough', 'Neighbourhood','Postcode'], axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(k)

In [128]:
centroid = final.mean(axis=0)
map_clusters = folium.Map(location=[centroid.Latitude, centroid.Longitude], zoom_start=11)

In [129]:
final.insert(0, 'Cluster', kmeans.labels_)

In [131]:
colors_array = cm.rainbow([0.1,0.2,0.7,0.8,0.9])
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [132]:
for lat, lon, poi, cluster in zip(final['Latitude'], final['Longitude'],
                                  final['Neighbourhood'], final['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)

In [133]:
map_clusters